In [1]:
import torch
import numpy as np
from qmc.tracehess import autograd_trace_hessian
from torch import nn, optim

In [34]:
class NelectronVander(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha_i*r_i)
    #input is 1D tensor which determines the number of particles (i.e. the dimension)
    def __init__(self, alpha):
        super(NelectronVander, self).__init__()
        self.alpha = nn.Parameter(alpha)
    
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(self.alpha.shape[0],self.alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.alpha.shape[0],self.alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[1] ])),-1)
    
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.prod(a[...,torch.triu(torch.ones(self.alpha.shape[0],self.alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.alpha.shape[0],self.alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    
        

In [27]:
class NelectronVanderWithMult(nn.Module):
    #ansatz given by the Vandermonde determinant of the one electron wavefunctions e^(-alpha_i * r_i) 
    #multiplied by e^(-beta * (r_1 + r_2 + ... r_N))
    #input is alpha, beta where alpha is 1D tensor which determines the number of particles and beta is scalar
    def __init__(self, alpha, beta):
        super(NelectronVanderWithMult, self).__init__()
        self.alpha = nn.Parameter(alpha)
        self.beta = nn.Parameter(beta)
    
    def forward(self, x):
        #returns the log prob. of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return 2 * ( -self.beta * torch.sum(x, -1)
            + torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(self.alpha.shape[0],self.alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.alpha.shape[0],self.alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[1] ])),-1) )
    
    
    def wave(self,x):
        # Returns the value of the wavefunction
        #input is tensor of size m x alpha.size or m x n x alpha.size
        a = torch.exp(-self.alpha*x.unsqueeze(-1)) - torch.exp(-self.alpha*x.unsqueeze(-2))
        return torch.exp(-self.beta * torch.sum(x, -1)) * torch.prod(a[...,torch.triu(torch.ones(self.alpha.shape[0],self.alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.alpha.shape[0],self.alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[1] ],-1)
    
    

In [41]:
f = NelectronVander(torch.rand(7))

In [42]:
f(torch.rand((6,4,7)))

tensor([[-140.2287, -158.8274, -148.6216, -131.0404],
        [-134.1732, -150.9533, -142.8486, -137.7906],
        [-157.4558, -149.6327, -143.4336, -149.1388],
        [-155.0732, -130.5947, -134.9153, -149.3318],
        [-146.7606, -164.3714, -141.2836, -144.3219],
        [-139.4302, -139.9384, -143.6349, -153.8013]], grad_fn=<MulBackward0>)

In [43]:
f.wave(torch.rand((3,7,7)))

tensor([[ 3.2181e-37, -1.0076e-33, -1.9495e-35,  4.9244e-31,  8.8114e-34,
          1.5578e-31,  2.9249e-30],
        [ 1.2741e-30,  5.7088e-30,  7.6091e-36, -2.6744e-33, -6.2717e-34,
         -9.6544e-32,  2.2516e-29],
        [-6.2991e-31,  1.3983e-31, -1.3969e-29,  1.2151e-31, -3.8522e-31,
          3.5215e-30, -2.4210e-32]], grad_fn=<ProdBackward1>)

In [44]:
x = torch.rand(4,5)

In [49]:
f = NelectronVanderWithMult(torch.rand(6),torch.rand(1))

In [50]:
f(torch.rand((9,6)))

tensor([-69.7354, -73.3459, -61.1760, -89.5785, -77.5670, -75.3314, -58.9335,
        -63.5868, -66.1187], grad_fn=<MulBackward0>)

In [51]:
f.wave(torch.rand((3,7,6)))

tensor([[ 1.3887e-16,  2.3443e-14, -5.6742e-25, -1.3181e-17,  7.7690e-16,
          2.3195e-20,  7.8268e-17],
        [ 2.1112e-16, -3.3355e-16, -4.7182e-16, -9.7660e-13,  5.6408e-16,
         -7.7414e-18, -3.8019e-14],
        [ 2.7246e-15, -1.4562e-16,  1.0129e-17,  2.7563e-16, -6.9796e-19,
          3.9876e-17, -2.1518e-18]], grad_fn=<MulBackward0>)

In [20]:
a = torch.exp(-2*x.unsqueeze(-1)) - torch.exp(-2*x.unsqueeze(-2))
alpha = torch.rand(5)

In [ ]:
torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(self.alpha.shape), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(self.alpha.shape), diagonal=1).nonzero(as_tuple = True)[1] ])),-1)


In [33]:
torch.sum(torch.log(torch.abs(a[...,torch.triu(torch.ones(alpha.shape[0],alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[0],torch.triu(torch.ones(alpha.shape[0],alpha.shape[0]), diagonal=1).nonzero(as_tuple = True)[1] ])),-1)

tensor([-13.8551, -23.7369, -15.4061, -12.1034])

In [22]:
torch.triu(torch.ones(alpha.shape), diagonal=1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [25]:
alpha.shape[0]

5